In [1]:
from subtitle_process import fetch_subtitle
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import getpass
import os
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [2]:
#从页面所有BV号构建url
urls = []
with open('url.json', 'r') as file:
    urls = json.load(file)

urls = ['https://'+item+'/' for item in urls]

In [3]:
#将前25个视频的字幕加入subtitles
subtitles = []
for url in urls[:5]:
    sub = fetch_subtitle(url)
    subtitles.append(sub)

len(subtitles)

Extracted segment: 13S411A73f
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/19058691521601443768adce9ddfec737839346a06b339dd1f2a?auth_key=1720061303-f20fd1f31ef046cdac7ec6d63e4cb433-0-6af2fa1aec4b31c43bc398b2762d4fa5
Extracted segment: 1Jf421Q7dF
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/12057938431597546258b9bb9856015fcb212780f97e227e686e?auth_key=1720061309-f10ae30081c84f4892a936448f0db72f-0-26dd637df40025b20787541dd13dbb28
Extracted segment: 1A1421k7JD
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/15559570961597535412b584fccc48b0e4d25ed9edcd9f4b72aa?auth_key=1720061312-8def080e871e41109e1237d68f787dd6-0-98d7e98733b40afe8db937ec088a0a49
Extracted segment: 17z421z7fx
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/135595662015965277167223118aa

5

In [4]:
len(subtitles)

5

In [10]:
template_sub = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一些相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

###原题：
（提取出的原题）

###讲解：
（提取出的讲解）

###相似题一：
（构造的相似题目一）

###相似题一讲解：
（构造的相似题目一的讲解）

###相似题二：
（构造的相似题目二）

###相似题二讲解：
（构造的相似题目二的讲解）

视频字幕：
{subtitle}

"""

In [7]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [13]:
'''
输入：原始视频字幕
输出：gpt从字幕中提取出的题干以及讲解
'''
async def inference(sub: str) -> str:
    
    output = ""
    llm = ChatOpenAI(
        model='gpt-4o',
        temperature=0,
        max_tokens=1000,
        max_retries=2,
        timeout=20,
    )

    prompt = PromptTemplate.from_template(template=template_sub)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    async for chunk in chain.astream({"subtitle": sub}, version="v2"):
        output += chunk

    return output

In [14]:
a = await inference(subtitles[0])
a

'### 原题：\n在一次羽毛球赛中，八个队进行循环赛，需要比赛几场？\n\n### 讲解：\n根据我们刚才老师给大家说的，如果你代表的是其中的一个队，你需要和其他七个队进行比赛，所以是七场。然后我们需要计算所有队伍之间的比赛场次，可以用等差数列的求和公式来计算。具体步骤如下：\n\n1. 第一个队需要和其他七个队比赛，所以是7场。\n2. 第二个队需要和剩下的六个队比赛，所以是6场。\n3. 以此类推，直到最后一个队不需要再比赛。\n\n所以总场次为：7 + 6 + 5 + 4 + 3 + 2 + 1 = 28场。\n\n我们可以用凑整法来快速计算：\n7 + 3 = 10\n6 + 4 = 10\n5 + 5 = 10\n2 + 1 = 3\n\n所以总和为：10 + 10 + 10 + 3 = 33场。\n\n### 相似题一：\n在一次乒乓球赛中，六个队进行循环赛，需要比赛几场？\n\n### 相似题一讲解：\n同样的道理，如果你代表的是其中的一个队，你需要和其他五个队进行比赛，所以是五场。然后我们需要计算所有队伍之间的比赛场次，可以用等差数列的求和公式来计算。具体步骤如下：\n\n1. 第一个队需要和其他五个队比赛，所以是5场。\n2. 第二个队需要和剩下的四个队比赛，所以是4场。\n3. 以此类推，直到最后一个队不需要再比赛。\n\n所以总场次为：5 + 4 + 3 + 2 + 1 = 15场。\n\n我们可以用凑整法来快速计算：\n5 + 1 = 6\n4 + 2 = 6\n3\n\n所以总和为：6 + 6 + 3 = 15场。\n\n### 相似题二：\n在一次篮球赛中，十个队进行循环赛，需要比赛几场？\n\n### 相似题二讲解：\n同样的道理，如果你代表的是其中的一个队，你需要和其他九个队进行比赛，所以是九场。然后我们需要计算所有队伍之间的比赛场次，可以用等差数列的求和公式来计算。具体步骤如下：\n\n1. 第一个队需要和其他九个队比赛，所以是9场。\n2. 第二个队需要和剩下的八个队比赛，所以是8场。\n3. 以此类推，直到最后一个队不需要再比赛。\n\n所以总场次为：9 + 8 + 7 + 6 + 5 + 4 + 3 + 2 + 1 = 45场。\n\n我们可以用凑整法来快速计算：\n9 + 1 = 10\n8 + 2 = 10\n7 + 3